# Project 2
This is project 2. I did stuff.

In [2]:
import numpy as np
import pydotplus
import pprint as pp
import mlrose
import pandas as pd
import random
import datetime

ModuleNotFoundError: No module named 'mlrose'

In [ ]:
# load local files
%run util/util
%run util/stats
%run util/charts

In [ ]:
max_trials = 10
rand_seeds = random.sample(range(1, 999999999), max_trials)

In [ ]:
def print_groups(g):
    for key, item in g:
        print(g.get_group(key), "\n\n")

In [ ]:
def show_aggregates(attempts):
    att_group = attempts.groupby('max-attemtps')
    fitness_agg = att_group['best-fit-score'].agg([np.mean, np.std, np.median, np.min, np.max])
    fitness_agg.columns = ['Fitness Mean', 'Fitness Std.', 'Fitness Median', 'Fitness Min', 'Fitness Max']
    time_agg = att_group['time'].agg([np.mean])
    time_agg.columns = ['Time Mean']
    data_agg_comb = pd.concat([fitness_agg, time_agg], axis=1, sort=False)
    return data_agg_comb

In [ ]:
def try_problem(prob_f, attempt_max):
    attempts = []
    for a in range(1, attempt_max + 1):
        for t in range(len(rand_seeds)):
            seed = rand_seeds[t]
            start_time = datetime.datetime.now()
            best_state, best_fit_score = prob_f(a, seed)
            total_delta = datetime.datetime.now() - start_time
            microseconds  = total_delta.seconds * 1000000 + total_delta.microseconds
            attempts.append([a, t, best_state, best_fit_score, seed, microseconds])
    df = pd.DataFrame(attempts, columns=['max-attemtps', 'trial-n', 'best-state', 'best-fit-score', 'seed', 'time'])
    
    return df

In [ ]:
def try_sim_ann(prob, initial_state, attempt_max):
    def p(attempts, seed):
        return mlrose.simulated_annealing(prob, schedule = mlrose.ExpDecay(), max_attempts = attempts, 
                                                              max_iters = 1000, init_state = initial_state, 
                                                              random_state = seed)
    return try_problem(p, attempt_max)

In [ ]:
def try_genetic(prob, attempt_max):
    def p(attempts, seed):
        return mlrose.genetic_alg(prob, max_attempts=attempts, random_state=seed)
    return try_problem(p, attempt_max)

In [ ]:
def try_rand_hill(prob, initial_state, attempt_max):
    def p(attempts, seed):
        return mlrose.random_hill_climb(prob, max_attempts=attempts, random_state=seed)
    return try_problem(p, attempt_max)

In [ ]:
def try_mimic(prob, attempt_max):
    def p(attempts, seed):
        return mlrose.mimic(prob, max_attempts=attempts, random_state=seed)
    return try_problem(p, attempt_max)

In [ ]:
def show_learning_rate(data, title, ylim=None, xlim=None):
    plt.plot(data['Fitness Mean'], 'b', data['Fitness Median'], 'r')
    plt.legend(['Mean Fitness', 'Median Fitness'], loc='lower right')
    plt.xlabel('iterations')
    plt.ylabel('fitness')
    if ylim is not None:
        plt.ylim(ylim[0],ylim[1])
    if xlim is not None:
        plt.xlim(xlim[0],xlim[1])
    plt.title(title)
    plt.show()

In [ ]:
def show_learning_time(data, title):
    plt.plot(data['Time Mean'], 'gray')
    plt.legend(['Time Mean'], loc='lower right')
    plt.xlabel('iterations')
    plt.ylabel('time')
    plt.title(title)
    plt.show()

In [ ]:
def show_charts(data, title):
    show_learning_rate(data, title, ylim=[15,29])
    show_learning_time(data, title + ' Time')

## n-queens
The `n-queens` problem is a classic problem for exploring genetic algorithms.

In [ ]:
# derived from https://en.wikipedia.org/wiki/Triangular_number and https://www.mathsisfun.com/algebra/triangular-numbers.html
def triangle_number(n):
    return n*(n+1)/2

def valid_queens_state(state):
    # used code from https://stackoverflow.com/questions/13252333/python-check-if-all-elements-of-a-list-are-the-same-type
    return all(isinstance(x, int) for x in state) and min(state) >= 0 and max(state) <= len(state) - 1

# this is a modified version of a similar function found here https://github.com/gkhayes/mlrose/blob/master/tutorial_examples.ipynb
def n_queens_fitness_fn(state):
    fitness = 0
    for i in range(len(state) - 1):
        for j in range(i + 1, len(state)):
            # Check for horizontal, diagonal-up and diagonal-down attacks
            if (state[j] != state[i]) and (state[j] != state[i] + (j - i)) and (state[j] != state[i] - (j - i)):
                fitness += 1

    return fitness

# modified https://solarianprogrammer.com/2017/11/20/eight-queens-puzzle-python/
def show_full_board(state):
    size = len(state)
    board = ""
    for row in range(size):
        line = ""
        for column in range(size):
            if state[row] == column:
                line += "* "
            else:
                line += ". "
        board = board + line + "\n"
    return board

In [ ]:
pp.pprint(show_full_board([1,3,5,7,2,0,6,4]))

In [ ]:
# run a few quick tests to make sure my functions are working
test_states = [{"s":[0,1,2,3], "v": True, "es": 0, "eb": "* . . . \n. * . . \n. . * . \n. . . * \n"},
               {"s":[1,2,3,4], "v": False},
               {"s":[0,1,2,3,4,5,6,7], "v": True, "es": 0, "eb": "* . . . . . . . \n. * . . . . . . \n. . * . . . . . \n. . . * . . . . \n. . . . * . . . \n. . . . . * . . \n. . . . . . * . \n. . . . . . . * \n"},
               {"s": [1,4,1,3,5,5,2,7], "v": True, "es": 26, "eb": ". * . . . . . . \n. . . . * . . . \n. * . . . . . . \n. . . * . . . . \n. . . . . * . . \n. . . . . * . . \n. . * . . . . . \n. . . . . . . * \n"},
               {"s":[1,3,5,7,2,0,6,4], "v": True, "es": 27, "eb": ". * . . . . . . \n. . . * . . . . \n. . . . . * . . \n. . . . . . . * \n. . * . . . . . \n* . . . . . . . \n. . . . . . * . \n. . . . * . . . \n"}]

for s in test_states:
    show_full_board(s["s"])
    if(valid_queens_state(s["s"]) != s["v"]):
        raise Exception("valid_queens_state is broken. Validity for " + str(s["s"]) + 
                        " was " + str(valid_queens_state(s["s"])) + " but should have been " + str(s["v"]))
    if(valid_queens_state(s["s"]) != s["v"] and n_queens_fitness(s["s"]) != s["es"]):
        raise Exception("n_queens_fitness is broken. Fitness for " + str(s["s"]) + 
                        " was " + str(n_queens_fitness(s["s"])) + " but should have been " + str(s["es"]))
    if(valid_queens_state(s["s"]) and show_full_board(s["s"]) != s["eb"]):
        raise Exception("show_full_board is broken. Fitness for " + str(s["s"]) + 
                        " was " + str(show_full_board(s["s"])) + " but should have been " + str(s["eb"]))
    

In [ ]:
best_posible_fitness = triangle_number(8 - 1)
n_queens_fitness = mlrose.CustomFitness(n_queens_fitness_fn)

In [ ]:
#define problem
n_queens_problem = mlrose.DiscreteOpt(length = 8, fitness_fn = n_queens_fitness, maximize = True, max_val = 8)

In [ ]:
n_queens_fitness_fn([1,3,0,0])

In [ ]:
n_queens_fitness_fn([0,0,3,1])

In [ ]:
n_queens_fitness_fn([1,3,3,1])

In [ ]:
n_queens_sim_ann = try_sim_ann(n_queens_problem, [0] * 8, 100)

In [ ]:
n_queens_sim_ann_agg = show_aggregates(n_queens_sim_ann)

In [ ]:
n_queens_sim_ann_agg.iloc[list(range(20,30))]

In [ ]:
show_charts(n_queens_sim_ann_agg, 'N-Queens Simulated Annealing')

In [ ]:
n_queens_genetic = try_genetic(n_queens_problem, 100)

In [ ]:
n_queens_genetic_agg = show_aggregates(n_queens_genetic)

In [ ]:
n_queens_genetic_agg.iloc[list(range(10,20))]

In [ ]:
show_charts(n_queens_genetic_agg, 'N-Queens Genetic Algorithm')

In [ ]:
n_queens_rand_hill = try_rand_hill(n_queens_problem, [0] * 8, 100)

In [ ]:
n_queens_rand_hill_agg = show_aggregates(n_queens_rand_hill)
show_charts(n_queens_rand_hill_agg, 'N-Queens Random Hill Climbling')

In [ ]:
n_queens_rand_hill_agg.iloc[list(range(30,40))]

In [ ]:
n_queens_mimic = try_mimic(n_queens_problem, 100)

In [ ]:
n_queens_mimic_agg = show_aggregates(n_queens_mimic)
show_charts(n_queens_mimic_agg, 'N-Queens MIMIC')

In [ ]:
n_queens_mimic_agg.iloc[list(range(0,10))]

In [ ]:
n_queens_mimic.iloc[list(range(990,1000))]